In [9]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import json
import glob
from os import walk

read_files = glob.glob("*.json")
output_list = []
for f in read_files:
    with open(f, "r") as infile:
        for line in infile:
            output_list.append(json.loads(line))
dfItem = pd.DataFrame.from_records(output_list)
print(dfItem)



               created_date   current_time in_reply_to_status_id_str  \
0       2018-06-14 08:49:04  1528958944022                      None   
1       2018-06-14 08:49:04  1528958944267                      None   
2       2018-06-14 08:49:04  1528958944282                      None   
3       2018-06-14 08:49:04  1528958944327                      None   
4       2018-06-14 08:49:04  1528958944345                      None   
...                     ...            ...                       ...   
401994  2018-06-14 15:43:53  1528983833387                      None   
401995  2018-06-14 15:43:53  1528983833533                      None   
401996  2018-06-14 15:43:53  1528983833536                      None   
401997  2018-06-14 15:43:53  1528983833598                      None   
401998  2018-06-14 15:43:53  1528983833600                      None   

        in_reply_to_status_id                      created_at  \
0                         NaN  Thu Jun 14 06:49:03 +0000 2018   
1    